# Stream+ Control Group Analysis - June 30, 2022 Campaign

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [ ]:
scrubbed_df = pd.read_csv('DB30184396_DSE_Check_20220630.csv')

In [ ]:
%%bigquery stream_plus_df

SELECT * FROM `cto-wln-sa-data-pr-bb5283.customer_personas_reports.TELUS_firefly_campaign_output`

In [ ]:
scrubbed_df.groupby(['TARGET_DESC', 'CONTROL_GROUP'])[['BAN']].count().reset_index()

In [ ]:
scrubbed_df.head()

In [ ]:
scrubbed_df.groupby(['TARGET_DESC', 'CONTROL_GROUP'])[['PROPENSITY_SCORE']].describe()

In [ ]:
measurements_df = pd.read_csv('Steam+_Takers_Detail_Full_Data_june30.csv')

In [ ]:
measurements_df.columns

In [ ]:
measurements_df.head()

In [ ]:
measurements_df['Campaign Description'].unique()

In [ ]:
scrubbed_df.groupby(['TARGET_DESC', 'CONTROL_GROUP'])[['BAN']].count().reset_index().pivot(index='TARGET_DESC', columns='CONTROL_GROUP', values='BAN').reset_index()

In [ ]:
sales_sum_df = measurements_df.groupby(['Stream+ Sales?', 'Target Desc', 'Test Group' ])['Subscriber'].count().reset_index()

In [ ]:
sales_sum_pivot_df = sales_sum_df.pivot(index='Target Desc', columns='Test Group', values='Subscriber').reset_index()

In [ ]:
sales_sum_pivot_merge_df = pd.merge(sales_sum_pivot_df, 
                                    scrubbed_df.groupby(['TARGET_DESC', 'CONTROL_GROUP'])[['BAN']].count().reset_index().pivot(index='TARGET_DESC', columns='CONTROL_GROUP', values='BAN').reset_index(),
                                    how='inner',
                                    left_on='Target Desc',
                                    right_on = 'TARGET_DESC'
                                   )

In [ ]:
sales_sum_pivot_merge_df

In [ ]:
sales_sum_pivot_merge_df['Control Group Sales Rate'] = sales_sum_pivot_merge_df['Control Group']/sales_sum_pivot_merge_df['Y']
sales_sum_pivot_merge_df['Target Group Sales Rate'] = sales_sum_pivot_merge_df['Target Group']/sales_sum_pivot_merge_df['N']

In [ ]:
sales_sum_pivot_merge_df

In [ ]:
sales_sum_pivot_merge_df['lift'] = (sales_sum_pivot_merge_df['Target Group Sales Rate'] - sales_sum_pivot_merge_df['Control Group Sales Rate'])/sales_sum_pivot_merge_df['Control Group Sales Rate'] * 100

In [ ]:
sales_sum_pivot_merge_df['msg_type'] = 'commuter'
sales_sum_pivot_merge_df['msg_type'] = sales_sum_pivot_merge_df['Target Desc'].apply(lambda x: 'streaming' if 'streaming' in x else 'commuter' )

In [ ]:
sales_sum_pivot_merge_df

In [ ]:
sales_sum_pivot_merge_2_df = sales_sum_pivot_merge_df.groupby('msg_type').sum().reset_index()

In [ ]:
sales_sum_pivot_merge_2_df

In [ ]:
sales_sum_pivot_merge_2_df['Control Group Sales Rate'] = sales_sum_pivot_merge_2_df['Control Group']/sales_sum_pivot_merge_2_df['Y']
sales_sum_pivot_merge_2_df['Target Group Sales Rate'] = sales_sum_pivot_merge_2_df['Target Group']/sales_sum_pivot_merge_2_df['N']
sales_sum_pivot_merge_2_df['lift'] = (sales_sum_pivot_merge_2_df['Target Group Sales Rate'] - sales_sum_pivot_merge_2_df['Control Group Sales Rate'])/sales_sum_pivot_merge_2_df['Control Group Sales Rate'] * 100

In [ ]:
.00399 - .00306

In [ ]:
sales_sum_pivot_merge_2_df.to_csv('stream_plus_june30_summary.csv', index=False)

In [ ]:
measurements_df.groupby('Control Group')['Subscriber'].count()

In [ ]:
scrubbed_df['adopted_stream_plus']=0
scrubbed_df.loc[scrubbed_df.SUBSCRIBER_NO.isin(measurements_df.Subscriber), 'adopted_stream_plus']=1

In [ ]:
scrubbed_df.groupby('CONTROL_GROUP')['adopted_stream_plus'].sum()

## Most recent firefly output

In [ ]:
stream_plus_df[['propensity_score']].describe()

In [ ]:
stream_plus_df.loc[stream_plus_df.propensity_score <= stream_plus_df.propensity_score.quantile(0.5)].shape

In [ ]:
stream_plus_df.loc[stream_plus_df.propensity_score >= stream_plus_df.propensity_score.quantile(0.75)].shape

In [ ]:
scrubbed_df[['PROPENSITY_SCORE']].describe()

In [ ]:
scrubbed_df.groupby(['TARGET_DESC', 'CONTROL_GROUP', 'adopted_stream_plus'])[['PROPENSITY_SCORE']].describe().reset_index().to_csv('stream_plus_june30_propensity_stats.csv',index=False)

In [ ]:
scrubbed_df['new_control_group'] = 'N'

In [ ]:
scrubbed_df.loc[scrubbed_df.SUBSCRIBER_NO.isin(scrubbed_df.loc[(scrubbed_df.PROPENSITY_SCORE < scrubbed_df.PROPENSITY_SCORE.quantile(q=0.6))].sample(n=22224).SUBSCRIBER_NO), 'new_control_group'] = 'Y'


In [ ]:
scrubbed_df.loc[(scrubbed_df.adopted_stream_plus==1) & (scrubbed_df.new_control_group=='Y')].PROPENSITY_SCORE.describe()

In [ ]:
scrubbed_df.groupby(['new_control_group', 'adopted_stream_plus'])['SUBSCRIBER_NO'].count()

In [ ]:
scrubbed_df.new_control_group.value_counts()

In [ ]:
scrubbed_df.PROPENSITY_SCORE.quantile(q=0.5)

In [ ]:
scrubbed_df.loc[(scrubbed_df.adopted_stream_plus==1) & (scrubbed_df.CONTROL_GROUP=='Y')].PROPENSITY_SCORE.describe()

In [ ]:
scrubbed_df.loc[(scrubbed_df.adopted_stream_plus==1) & (scrubbed_df.CONTROL_GROUP=='Y') & (scrubbed_df.PROPENSITY_SCORE<1.69)].shape

In [ ]:
scrubbed_df.loc[(scrubbed_df.adopted_stream_plus==1) & (scrubbed_df.CONTROL_GROUP=='N')].PROPENSITY_SCORE.describe()